In [50]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import random as r

initdf = pd.read_csv('cpidata_whole.csv')
initdf.head()

datelist = [initdf['date'][i].split('/') for i in range(len(initdf))]
initdf['year'] = [int(datelist[i][2]) for i in range(len(initdf))]


list1 = ['dairyinf', 'mpfeinf', 'fvinf']
list2 = ['dairycpi', 'mpfecpi', 'fvcpi']

for i in range(len(list1)):
    initdf[list1[i]] = initdf[list2[i]].pct_change(periods=1)


initdf.dropna(inplace=True)
initdf.reset_index()
initdf


filterlist = [r.randint(1967,2020) for i in range(11)]
summarydict = {}


for year in filterlist:
    currdf = initdf[(initdf['year'] >= year) & (initdf['year'] <= (year + 4))]
    for i in range(len(list1)):
        summarydict[str(year)+list1[i]] = {'mean':round(float(currdf[list1[i]].mean()), 2),
                            'median':currdf[list1[i]].median(),
                            'mode':round(float(currdf[list1[i]].mode()[0]),2),
                            'standard error':round(float(currdf[list1[i]].sem()),2)}
    
    for item in list1:
        fig, ax = plt.subplots()
        ax.plot(currdf['date'], currdf[item])
        ax.set_xticks(ax.get_xticks()[::2])
        fig.tight_layout()
        plt.xlabel(f'Date')
        plt.ylabel(f'% Change Inflation')
        plt.title(f'% Change Inflation from {year} to {year+4} for {item}')
        plt.xticks(fontsize = 5, rotation=90)
        plt.show()



,date,dairycpi,mpfecpi,fvcpi,year,dairyinf,mpfeinf,fvinf
444,1/1/2004,172.400,180.000,225.900,2004,-0.003468,-0.003874,-0.007033
445,2/1/2004,172.100,179.900,227.000,2004,-0.001740,-0.000556,0.004869
446,3/1/2004,171.900,179.400,229.900,2004,-0.001162,-0.002779,0.012775
447,4/1/2004,174.000,179.900,229.100,2004,0.012216,0.002787,-0.003480
448,5/1/2004,185.900,182.000,230.000,2004,0.068391,0.011673,0.003928
449,6/1/2004,188.800,182.500,229.000,2004,0.015600,0.002747,-0.004348
450,7/1/2004,187.700,183.600,228.100,2004,-0.005826,0.006027,-0.003930
451,8/1/2004,184.900,183.300,229.400,2004,-0.014917,-0.001634,0.005699
452,9/1/2004,181.600,182.600,229.200,2004,-0.017847,-0.003819,-0.000872
453,10/1/2004,182.100,182.500,239.800,2004,0.002753,-0.000548,0.046248
